# Traffic Analysis
<i>Adapted for use with FABRIC from [Traffic Analysis](https://www.cs.unc.edu/Research/geni/geniEdu/05-TrafficAnalysis.html)
    
In this tutorial you will learn a variety of tools for traffic analysis, with a focus on ping and tcpdump.
    
<b> Prerequisites  
    
* You need to have your FABRIC bastion host key pair set up to do this tutorial. If you have not already set this up, follow steps 1-3 at https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/.
* You are comfortable using ssh and executing basic commands using a UNIX shell. [Tips about how to login to hosts.](https://learn.fabric-testbed.net/knowledge-base/logging-into-fabric-vms/)

## 1. Design the Experiment
### 1.1 Reserve Resources
A topology with 2 nodes (a client and a server) will be reserved.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

import json
import traceback

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name="TrafficAnalysis")
    
    #Server
    server = slice.add_node(name="server", site="MAX")
    server.set_image('default_ubuntu_20')
    server_port = server.add_component(model='NIC_Basic', name="server_port").get_interfaces()[0] 
    
    #Client
    client = slice.add_node(name="client", site="TACC")
    client.set_image('default_ubuntu_20')
    client_port = client.add_component(model='NIC_Basic', name="client_port").get_interfaces()[0] 
    
    lan = slice.add_l2network(name="lan", interfaces=[server_port, client_port])
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Slice Failed: {e}")

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

try:    
    server = slice.get_node(name="server") 
    client = slice.get_node(name="client")
    
    subnet1 = IPv4Network("10.1.1.0/24")
    
    server_port = server.get_interface(network_name="lan") 
    server_port.ip_addr_add(addr="10.1.1.2", subnet=subnet1)
    
    client_port = client.get_interface(network_name="lan") 
    client_port.ip_addr_add(addr="10.1.1.1", subnet=subnet1)
except Exception as e:
    print(f"Exception: {e}")

### 1.3 Installation
We will be installing net-tools package on both the server and client nodes, containing many useful networking tools.

In [ ]:
for node in slice.get_nodes():
    node.execute("sudo apt install net-tools")

## 2. Experiment
### 2.1 ifconfig
1. On the client run:
<br>```ifconfig```
<br> This command lists the interfaces and their status. The interface facing the server is 'ens7' with the IP address of '10.1.1.2'.


### 2.2 Adjust the MTU
Maximum Transmission Unit (MTU) is a value representing the maximum length data packet that a a device (NIC) can accept.
1. On the server SSH terminal run:
<br>```ifconfig```

2. The MTU size can be adjusted with the following command, where 'interface name' corresponds to the interface facing the client:
<br>```sudo ifconfig <interface name> mtu 1400```
3. Repeat steps 1 and 2 on the client terminal.


### 2.3 Ping
Ping is one of the most popular networking tools, inheriting the name from sonar technology. The ping tool sends packets, and gathers information from their "echos". Such information includes determining if  a host is reachable and the round-trip time (RTT) to that host.
1. On the client terminal run:
<br>```ping 10.1.1.2```
<br>ICMP echo request packets are sent to the target host, and the host will generate ICMP responses. With the response the originating host calculates the RTT and detects lost packets.

2. Press Ctrl + C to stop pinging.
3. Ping can be verified by bringing down the connection, to do so run on the interface connected to the server (ens7):
<br>```sudo ip link set dev <interface name> down```
4. Ping the server again:
<br>```ping 10.1.1.2```
<br>Take note of the change in RTT
5. Bring the connection back up by running:
<br>```sudo ip link set dev <interface name> up```
6. Ping the server again:
<br>```ping 10.1.1.2```

### 2.4 Tcpdump
1. On the client terminal run:
<br>```sudo tcpdump -i <interface name> -nn```
<br>Where ```-i <interface name>``` is the previously used interface used to connect to the server node and -nn causes tcpdump to use the numerical IP addresses rather than using DNS to resolve the symbolic addresses. 
   
2. On the server terminal run:
<br>```ping 10.1.1.1```

3. Return to the client terminal and notice incoming packets looking similar to:
<br>```IP 10.1.1.2 > 10.1.1.1: ICMP echo request, id 2, seq 1, length 64```
<br>Each line corresponds to a packet travelling over the interface. The columns in the output correspond to:
    * Timestamp
    * Source IP
    * Destination IP
    * Layer 3 protocol name and associated information
4. Press ctrl + C on the client and server terminals to stop pinging and tcpdump.
5. On the client terminal run:
<br>```sudo tcpdump -i <interface name> -nn -S tcp```
<br> The addition of ```-S``` instructs tcpdump to print the absolute sequence numbers instead of the smaller, more human readable version. ```tcp``` filters the traffic to only show TCP data.
6. In the server terminal run:
<br>```nc -l 44444```
<br> This command has the netcat tool listen on port ```44444```. Netcat is a tool allowing for reading and writing both TCP and UDP traffic across a network.
7. Open an additional terminal on the client and run:
<br>```nc 10.1.1.2 44444```
<br>On the first client terminal (running tcpdump) you can see messages similar to:
<br>```10:26:33.508856 IP 10.1.1.1.46520 > 10.1.1.2.44444: Flags [S], seq 1812506012, win 14600, options [mss 1460,sackOK,TS val 1375289021 ecr 0,nop,wscale 7], length 0```
<br>```10:26:33.508939 IP 10.1.1.2.44444 > 10.1.1.1.46520: Flags [S.], seq 3211754307, ack 1812506013, win 14480, options [mss 1460,sackOK,TS val 1375289021 ecr 1375289021,nop,wscale 7], length 0```
<br>```10:26:33.508958 IP 10.1.1.1.46520 > 10.1.1.2.44444: Flags [.], ack 3211754308, win 115, options [nop,nop,TS val 1375289021 ecr 1375289021], length 0```
<br>These lines show the 3-way TCP handshake, initializing the connection between the server and the client. 
8. On the second client terminal (not running tcpdump) type:
<br>```Hello World```
<br>The netcat tool creates a TCP packet with this message and sends it to the server node.
<br>On the client terminal running tcpdump you should see messages similar to:
<br>```10:28:13.538448 IP 10.1.1.1.46520 > 10.1.1.2.44444: Flags [P.], seq 1812506013:1812506025, ack 3211754308, win 115, options [nop,nop,TS val 1375389051 ecr 1375289021], length 12```
<br>```10:28:13.538530 IP 10.1.1.2.44444 > 10.1.1.1.46520: Flags [.], ack 1812506025, win 114, options [nop,nop,TS val 1375389051 ecr 1375389051], length 0```
<br> The first packet is the outgoing ```Hello World``` message, while the second packet is the acknowledgement (ACK).
9. On the server terminal type:
<br>```Hello World```
<br>On the client terminal running tcpdump, messages similar to the previous step should be present and sent in the opposite direction.
10. Press ctrl + C on all terminals to close out of the running programs.

## 3. Cleanup Resources
### 3.1 Delete Slice

In [ ]:
try:
    slice = fablib.get_slice("TrafficAnalysis")
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")